In [1]:
# Change directory to the root folder of Dispa-SET:
import os
import sys
os.chdir('..')
import numpy as np
import pandas as pd
import datetime as dt
from dispaset.postprocessing import postprocessing as post
import importlib
importlib.reload(post)
%matplotlib inline
#Import Dispa-SET
import dispaset as ds
import matplotlib.pyplot as plt
from matplotlib.pyplot import *
import seaborn as sns
import matplotlib.dates as mdates
#plt.style.use('seaborn-colorblind')
import pickle
import time as tm
import logging
import matplotlib

In [2]:
%matplotlib inline
import matplotlib
matplotlib.rcParams['figure.figsize'] = [16, 8]

In [3]:
pd.options.display.max_rows = 60   #default 60 (None)
pd.options.display.max_columns = 60   #default 60 (None)
np.set_printoptions(threshold=np.inf)   #default 1000 (np.inf)

In [4]:
try:
    import cPickle as pickle
except ImportError:
    import pickle

In [5]:
def FindNetPowerFlow(PowerFlow):
    NewOutputFlow = PowerFlow.copy()
    OriginalColumns = []
    reverseindex = []
    for j,column in enumerate(NewOutputFlow):
        A = column.split(' -> ')
        OriginalColumns.append(A)
        B = list(reversed(A))
        if B in OriginalColumns:
            reverseindex.append(j)
            D = [i for i, C in enumerate(OriginalColumns) if C==B]
            D = int(''.join(map(str,D)))
            NewOutputFlow.iloc[:,D] = NewOutputFlow.iloc[:,D] - NewOutputFlow.iloc[:,j]
    NewOutputFlow = NewOutputFlow.drop(NewOutputFlow.columns[reverseindex], axis=1).copy()
    return NewOutputFlow

In [6]:
def FindUniquePowerExchange(PowerExchange):
    NewPowerExchange = PowerExchange.copy()
    Combinations = []
    reverseindex = []

    for zone1 in NewPowerExchange.columns.unique(level=0):
        for zone2 in NewPowerExchange.columns.unique(level=1):
            if zone1!=zone2:
                List = [zone1,zone2]
            if List not in Combinations:
                try:
                    NewPowerExchange.drop((zone2,zone1), axis = 1, inplace = True)
                except:
                    pass
                Combinations.append(List)
                Combinations.append(list(reversed(List)))
    return NewPowerExchange

In [7]:
from dispaset.postprocessing import postprocessing as post
import importlib
importlib.reload(post)

<module 'dispaset.postprocessing.postprocessing' from '/Users/abdullahalawad/Dropbox (MIT)/CCES/2017/Energy/Regional trade/UC&ED/dispaset/PycharmProjects/UC Model/dispaset/postprocessing/postprocessing.py'>

##  Test (build & run)

In [8]:
# Load the configuration file
config = ds.load_config_excel('Simulations/ConfigKSA.xlsx')

[INFO    ] (load_config_excel): Using config file Simulations/ConfigKSA.xlsx to build the simulation environment
[INFO    ] (load_config_excel): Using /Users/abdullahalawad/Dropbox (MIT)/CCES/2017/Energy/Regional trade/UC&ED/dispaset/PycharmProjects/UC Model/Simulations/simulation_KSA as simulation folder


In [9]:
# Build the simulation environment:
SimData, FuelPrices, FuelPrices2 = ds.build_simulation(config, LocalSubsidyMultiplier=1, ExportCostMultiplier=1)

[INFO    ] (build_simulation): New build started. DispaSET version: b'v2.3-9-g2ff8285'
[INFO    ] (NodeBasedTable): No data file found for the table LoadShedding. Using default value 0.0
[INFO    ] (NodeBasedTable): No data file found for the table CostLoadShedding. Using default value 1000.0
[INFO    ] (check_units): The columns "Nunits" is not present in the power plant database. A value of one will be assumed by default
[WARNING ] (build_simulation): No data file or default value found for PriceOfBlackCoal 2 in the zone SA_NEOA. Assuming zero marginal price!
[WARNING ] (interconnections): Zone SA_NWOA does not appear to be connected to any other zone in the NTC table. It should be simulated in isolation
[INFO    ] (clustering): Clustered 651 original units into 36 new units
[INFO    ] (build_simulation): Time to cluster power plants: 21.302369832992554s
[WARNING ] (build_simulation): TimeUpMinimum in the power plant data has been rounded to the nearest integer value
[WARNING ] (buil

In [10]:
r = ds.solve_GAMS(config['SimulationDirectory'], config['GAMS_folder'])

[INFO    ] (solve_GAMS): Using the low-level gams api
[WARNING ] (get_gams_path): The provided path for GAMS ( ) does not exist. Trying to locate...
[INFO    ] (get_gams_path): Detected /Applications//GAMS27.2/sysdir as GAMS path on this computer
[INFO    ] (solve_low_level): Completed simulation in 247.54 seconds
